<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import numpy as np
import random
import sys
import requests
import os
import datetime

In [2]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)

In [3]:
r.encoding = r.apparent_encoding

data = r.text

In [4]:
data[:100]

'Project Gutenberg’s The Complete Works of William Shakespeare, by William Shakespeare\r\n\r\nThis eBook '

In [5]:
data = data.split('\r\n')

In [6]:
toc = [l.strip() for l in data[44:130:2]]

In [7]:
# Skip the Table of Contents
data = data[135:]

In [8]:
toc

['THE TRAGEDY OF ANTONY AND CLEOPATRA',
 'AS YOU LIKE IT',
 'THE COMEDY OF ERRORS',
 'THE TRAGEDY OF CORIOLANUS',
 'CYMBELINE',
 'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK',
 'THE FIRST PART OF KING HENRY THE FOURTH',
 'THE SECOND PART OF KING HENRY THE FOURTH',
 'THE LIFE OF KING HENRY THE FIFTH',
 'THE FIRST PART OF HENRY THE SIXTH',
 'THE SECOND PART OF KING HENRY THE SIXTH',
 'THE THIRD PART OF KING HENRY THE SIXTH',
 'KING HENRY THE EIGHTH',
 'KING JOHN',
 'THE TRAGEDY OF JULIUS CAESAR',
 'THE TRAGEDY OF KING LEAR',
 'LOVE’S LABOUR’S LOST',
 'THE TRAGEDY OF MACBETH',
 'MEASURE FOR MEASURE',
 'THE MERCHANT OF VENICE',
 'THE MERRY WIVES OF WINDSOR',
 'A MIDSUMMER NIGHT’S DREAM',
 'MUCH ADO ABOUT NOTHING',
 'THE TRAGEDY OF OTHELLO, MOOR OF VENICE',
 'PERICLES, PRINCE OF TYRE',
 'KING RICHARD THE SECOND',
 'KING RICHARD THE THIRD',
 'THE TRAGEDY OF ROMEO AND JULIET',
 'THE TAMING OF THE SHREW',
 'THE TEMPEST',
 'THE LIFE OF TIMON OF ATHENS',
 'THE TRAGEDY OF TITUS ANDRONICUS',
 'THE HI

In [9]:
len(toc)

43

In [10]:
# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

In [11]:
locations = {
    id_:{
        'title':title, 'start':-99
    } for id_,title in enumerate(toc)}

In [12]:
# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})

In [13]:
import pandas as pd
df_toc = pd.DataFrame.from_dict(locations, orient='index')

In [14]:
df_toc

,title,start
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99
1,AS YOU LIKE IT,14380
2,THE COMEDY OF ERRORS,17172
3,THE TRAGEDY OF CORIOLANUS,20373
4,CYMBELINE,30347
5,"THE TRAGEDY OF HAMLET, PRINCE OF DENMARK",30365
6,THE FIRST PART OF KING HENRY THE FOURTH,37052
7,THE SECOND PART OF KING HENRY THE FOURTH,41768
8,THE LIFE OF KING HENRY THE FIFTH,-99
9,THE LIFE OF KING HENRY V,45177


In [15]:
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)

In [16]:
df_toc.loc[42, 'end'] = len(data)

In [17]:
df_toc.tail()

,title,start,end
38,THE PASSIONATE PILGRIM,164930,165169.0
39,THE PHOENIX AND THE TURTLE,165170,165263.0
40,THE RAPE OF LUCRECE,165264,167482.0
41,VENUS AND ADONIS,167483,169306.0
42,,169307,169308.0


In [18]:
df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [19]:
df_toc.head()

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379.0,
1,AS YOU LIKE IT,14380,17171.0,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372.0,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346.0,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364.0,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...


In [20]:
# divide b/w plays and sonets
sonets = data[:2776]
plays = data[2777:]

In [21]:
data[0]

'                    1'

In [22]:
def long_lines(lst_ln):
    clean = []
    
    for ln in lst_ln: 
        
        if len(ln) == 0:
            pass
        else:
            pct = len(ln.strip(' ')) / len(ln)

            if pct >= .5:
                clean.append(ln.lstrip())

    return clean

In [23]:
# May Not be Needed
sonets = long_lines(sonets)
plays = long_lines(plays)

## Word Encoding

This is just a start, and is not complete yet. 

In [24]:
vocab = list(set("\r\n".join(plays).split()))
words = [line.split() for line in plays]

In [25]:
len(vocab)

75578

## Character Encoding

Using the technique shown in lecture. 

In [26]:
text = '\r\n'.join(sonets)

chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

print(f"Our corpus contains {len(chars)} unique characters.")

Our corpus contains 63 unique characters.


In [27]:
# Create the Sequence Data

maxlen = 150
step = 1

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 95704


In [28]:
import numpy as np

# Specify x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [29]:
x.shape

(95704, 150, 63)

In [30]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), dropout=0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               327680    
_________________________________________________________________
dense (Dense)                (None, 63)                16191     
Total params: 343,871
Trainable params: 343,871
Non-trainable params: 0
_________________________________________________________________


In [32]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [33]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model.fit(x, y,
          batch_size=1024,
          validation_split=.2,
          epochs=100,
          callbacks=[print_callback, 
                     #EarlyStopping(min_delta=.02, monitor='val_loss', patience=10),
                     tensorboard_callback])

Epoch 1/100
 1/75 [..............................] - ETA: 0s - loss: 4.1417WARNING:tensorflow:From /Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
75/75 [==============================] - ETA: 0s - loss: 3.1806
----- Generating text after Epoch: 0
----- Generating with seed: " thou harder hast engrossed,
Of him, my self, and thee I am forsaken,
A torment thrice three-fold thus to be crossed:
Prison my heart in thy steel "
 thou harder hast engrossed,
Of him, my self, and thee I am forsaken,
A torment thrice three-fold thus to be crossed:
Prison my heart in thy steel ebv
hf(ewgmetm,s,hi tne itndwlnwenuturi? w mNsutryohe se tt drfs rw y   raf adowwnr d
ldamtnaeior sesnmehliie rem g‘nw s,eghcawtahtom  v aesiccouwrV wmhs e dnbrnt
rhu p  heos Io
 iy ehtfr,sebhmeian tkf ’fw de

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN